The MOT history API gives authorised organisations, businesses and individuals a way to access vehicle and MOT test information for vehicles registered in Great Britain and Northern Ireland.

The API is based on REST principles. The response data is in JSON format and uses standard HTTP error response codes.

The API uses OAuth 2.0 with the client credentials flow for authentication and authorisation.

When you have completed the registration process you will receive:

a client ID
a client secret
a scope URL
an access token URL
an API key

In [3]:
#
This code first obtains an authorization code that lasts for up to 60 minutes and caches this in a file.
Each subsequent call to the API will use the cached token until it expires, and then it generates a new one.
"""

SyntaxError: unterminated triple-quoted string literal (detected at line 4) (1100545653.py, line 4)

In [4]:
import requests
import tempfile
import json
import pandas as pd
import os
import pprint
import time

SECRET_INFO_FILE_NAME = "API_INFO.json"  # File to store the secret information
SECRET_INFO_PATH_NAME = 'C:\APIINFO'  # File to store the secret information

# Configuration
AUTHORITY = "https://login.microsoftonline.com/common"                              # Update as per your tenant
SCOPE = ["https://tapi.dvsa.gov.uk/.default"]                                       # Define the API scope
MOT_HISTORY_URL = "https://history.mot.api.gov.uk/v1/trade/vehicles/registration/"  # URL for MOT history API
TOKEN_CACHE_FILE_NAME = "token_cache.json"                                          # Cache file for token - to avoid re-authentication every time

In [5]:
# oBtain the secret information from the file
SECRET_FILE_PATH = os.path.join(SECRET_INFO_PATH_NAME, SECRET_INFO_FILE_NAME)

with open(SECRET_FILE_PATH, "r") as file:
    data = json.load(file)
    CLIENT_ID=data["CLIENT_ID"]
    CLIENT_SECRET=data["CLIENT_SECRET"]
    API_KEY=data["API_KEY"]
    TOKEN_URL=data["TOKEN_URL"]


In [48]:
# Function to fetch temp folder where the token cache file will be stored
def getTempFolder():
    return tempfile.gettempdir()

# Function to fetch the token cache file path
def getTokenTempCacheFilePathName():
    tempFolder=getTempFolder()
    return os.path.join(tempFolder, TOKEN_CACHE_FILE_NAME)  # Cache file for token - to avoid re-authentication  every time

# Function to fetch the access token    
def getAccessToken():
    token_cache_file_path = getTokenTempCacheFilePathName()
    token = None
    if os.path.exists(token_cache_file_path):
        with open(token_cache_file_path, "r") as file:
            data = json.load(file)
            if data["expiry"] > time.time():
                return data["access_token"]
            
            if token is None or time.time() > token["expiry"]:
                data ={"access_token": access_token,"expiry": time.time() + expires_in - 60}  # Subtract 60 seconds as a buffer from server expiry
                
                with open(token_cache_file_path, "w") as file:
                      json.dump(token, file)
                return token["access_token"]   

In [49]:


# Define the path to the cache file in the temp folder

def get_cached_token():
    """
    Retrieve the cached token from a file if it exists and is not expired.
    """
    
    try:
        TOKEN_TEMP_CACHE_FILE = getTokenTempCacheFilePathName()
        with open(TOKEN_TEMP_CACHE_FILE, "r") as file:
            data = json.load(file)
            if data["expiry"] > time.time():
                return data["access_token"]
    except (FileNotFoundError, KeyError, ValueError):
        pass
    return None


In [50]:

def save_token_to_cache(access_token, expires_in):
    """
    Save the token and its expiry time to a cache file.
    """
    TOKEN_TEMP_CACHE_FILE = getTokenTempCacheFilePathName()
    data = {
        "access_token": access_token,
        "expiry": time.time() + expires_in - 60,  # Subtract 60 seconds as a buffer from server expiry
    }
    with open(TOKEN_TEMP_CACHE_FILE, "w") as file:
        json.dump(data, file)


In [51]:

def get_access_token():
    """
    Retrieve a valid access token, either from the cache or by requesting a new one.
    """
    # Check the cached token has NOT expired
    token = get_cached_token()
    if token:
        return token, True

    # Request a new token
    payload = {
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "scope": SCOPE,
    }
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    response = requests.post(TOKEN_URL, data=payload, headers=headers)

    if response.status_code == 200:
        token_data = response.json()
        save_token_to_cache(token_data["access_token"], token_data["expires_in"])
        return token_data["access_token"], False
    else:
        raise Exception(f"Failed to fetch access token: {response.status_code} - {response.text}")



In [52]:


def get_mot_history(access_token,registration):
    """
    Query the MOT history API for a given vehicle registration.
    """
   
    headers = {
        "Authorization": f"Bearer {access_token}",
        "x-api-key": API_KEY,
    }
    url = MOT_HISTORY_URL + registration

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        #with open(TEMP_JSON_FILE, "w") as file1:
        #    json.dump(response.json(), file1)
        
        return response.json()
    else:
        raise Exception(
            f"Failed to fetch MOT history: {response.status_code} - {response.text}"
        )



In [53]:
registration = "EA12HLK"
r=registration.strip().upper()
#input("Enter vehicle registration number: ").strip().upper()        
        # Get access token
access_token, isCached = get_access_token()
                # Fetch MOT history
mot_history = get_mot_history(access_token, r)



if isCached:
    stat="cached"
else:
    stat="online"

print("used " + stat + " token")
print(r)
print("----------------------------------------------------------------------")

pp = pprint.PrettyPrinter()

pp.pprint(mot_history)


used online token
EA12HLK
----------------------------------------------------------------------
{'engineSize': '2143',
 'firstUsedDate': '2012-03-30',
 'fuelType': 'Diesel',
 'hasOutstandingRecall': 'Unknown',
 'make': 'MERCEDES-BENZ',
 'manufactureDate': '2012-03-30',
 'model': 'C',
 'motTests': [{'completedDate': '2024-09-24T11:14:43.000Z',
               'dataSource': 'DVSA',
               'defects': [],
               'expiryDate': '2025-10-02',
               'motTestNumber': '563751125728',
               'odometerResultType': 'READ',
               'odometerUnit': 'MI',
               'odometerValue': '111594',
               'testResult': 'PASSED'},
              {'completedDate': '2023-09-30T13:42:35.000Z',
               'dataSource': 'DVSA',
               'defects': [],
               'expiryDate': '2024-10-02',
               'motTestNumber': '526227857792',
               'odometerResultType': 'READ',
               'odometerUnit': 'MI',
               'odometerValue': 

In [1]:
from flask import Flask, jsonify, render_template_string

app = Flask(__name__)

# Example JSON data
data = {
    "engineSize": "2143",
    "firstUsedDate": "2012-03-30",
    "fuelType": "Diesel",
    "hasOutstandingRecall": "Unknown",
    "make": "MERCEDES-BENZ",
    "manufactureDate": "2012-03-30",
    "model": "C",
    "motTests": [
        {
            "completedDate": "2024-09-24T11:14:43.000Z",
            "dataSource": "DVSA",
            "defects": [],
            "expiryDate": "2025-10-02",
            "motTestNumber": "563751125728",
            "odometerResultType": "READ",
            "odometerUnit": "MI",
            "odometerValue": "111594",
            "testResult": "PASSED"
        },
        {
            "completedDate": "2023-09-30T13:42:35.000Z",
            "dataSource": "DVSA",
            "defects": [],
            "expiryDate": "2024-10-02",
            "motTestNumber": "526227857792",
            "odometerResultType": "READ",
            "odometerUnit": "MI",
            "odometerValue": "102557",
            "testResult": "PASSED"
        },
        # Add the remaining motTests here...
    ]
}

@app.route("/")
def index():
    # HTML template for displaying the JSON data
    html = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>JSON Data Viewer</title>
        <script>
            function toggleDetails(id) {
                const details = document.getElementById(id);
                details.style.display = details.style.display === "none" ? "block" : "none";
            }
        </script>
    </head>
    <body>
        <h1>JSON Data Viewer</h1>
        <pre>{{ data|tojson(indent=2) }}</pre>
        <h2>MOT Tests</h2>
        <ul>
            {% for test in data['motTests'] %}
                <li>
                    <strong>{{ test['completedDate'] }} - Result: {{ test['testResult'] }}</strong>
                    <button onclick="toggleDetails('details-{{ loop.index0 }}')">Toggle Details</button>
                    <div id="details-{{ loop.index0 }}" style="display: none;">
                        <p><strong>Odometer:</strong> {{ test['odometerValue'] }} {{ test['odometerUnit'] }}</p>
                        <p><strong>Defects:</strong></p>
                        <ul>
                            {% for defect in test['defects'] %}
                                <li>{{ defect['text'] }} ({{ defect['type'] }})</li>
                            {% else %}
                                <li>No defects</li>
                            {% endfor %}
                        </ul>
                    </div>
                </li>
            {% endfor %}
        </ul>
    </body>
    </html>
    """
    return render_template_string(html, data=data)

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

c:\Users\gwied\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


from flask import Flask, jsonify, render_template_string

app = Flask(__name__)

# Example JSON data
data = {
    "engineSize": "2143",
    "firstUsedDate": "2012-03-30",
    "fuelType": "Diesel",
    "hasOutstandingRecall": "Unknown",
    "make": "MERCEDES-BENZ",
    "manufactureDate": "2012-03-30",
    "model": "C",
    "motTests": [
        {
            "completedDate": "2024-09-24T11:14:43.000Z",
            "dataSource": "DVSA",
            "defects": [],
            "expiryDate": "2025-10-02",
            "motTestNumber": "563751125728",
            "odometerResultType": "READ",
            "odometerUnit": "MI",
            "odometerValue": "111594",
            "testResult": "PASSED"
        },
        {
            "completedDate": "2023-09-30T13:42:35.000Z",
            "dataSource": "DVSA",
            "defects": [],
            "expiryDate": "2024-10-02",
            "motTestNumber": "526227857792",
            "odometerResultType": "READ",
            "odometerUnit": "MI",
            "odometerValue": "102557",
            "testResult": "PASSED"
        },
        # Add the remaining motTests here...
    ]
}

@app.route("/")
def index():
    # HTML template for displaying the JSON data
    html = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>JSON Data Viewer</title>
        <script>
            function toggleDetails(id) {
                const details = document.getElementById(id);
                details.style.display = details.style.display === "none" ? "block" : "none";
            }
        </script>
    </head>
    <body>
        <h1>JSON Data Viewer</h1>
        <pre>{{ data|tojson(indent=2) }}</pre>
        <h2>MOT Tests</h2>
        <ul>
            {% for test in data['motTests'] %}
                <li>
                    <strong>{{ test['completedDate'] }} - Result: {{ test['testResult'] }}</strong>
                    <button onclick="toggleDetails('details-{{ loop.index0 }}')">Toggle Details</button>
                    <div id="details-{{ loop.index0 }}" style="display: none;">
                        <p><strong>Odometer:</strong> {{ test['odometerValue'] }} {{ test['odometerUnit'] }}</p>
                        <p><strong>Defects:</strong></p>
                        <ul>
                            {% for defect in test['defects'] %}
                                <li>{{ defect['text'] }} ({{ defect['type'] }})</li>
                            {% else %}
                                <li>No defects</li>
                            {% endfor %}
                        </ul>
                    </div>
                </li>
            {% endfor %}
        </ul>
    </body>
    </html>
    """
    return render_template_string(html, data=data)

if __name__ == "__main__":
    app.run(debug=True)


In [56]:
print(mot_history['registration'])

EA12HLK


In [59]:
import pandas as pd
import os
#type(mot_history)
#k=mot_history.keys()
#m=mot_history.get("motTests")
#print(m)
OUTPUTDICTIONARY={}
xn=-1
for x, y in mot_history.items():  # iterate over the top level dictionary
     if type(y)==str:         # ADD TO OUTPUT DICTIONARY IF VALUE IS STRING  
      OUTPUTDICTIONARY[x]=y               
     if type(y)==list:        # IF VALUE IS LIST, EACH LIST ELEMENT IS A DICT OBJECT SO WE ITERATE AGAIN OVER EACH DICT IN THE LIST
      motTestCount=len(y)
      for yy in y:
          xn=xn+1
          motTestCounter=motTestCount-xn
          if type(yy)==dict:
           an=0
           for a, b in yy.items(): # ITERATE OVER DICT
              if type(b)==str:
                 OUTPUTDICTIONARY[x+str(motTestCounter)+"." + a]=b # ADD TO OUTPUT DICTIONARY              
              elif type(b)==list:
                 for bb in b:
                   an=an+1
                   if type(bb)==str:
                        OUTPUTDICTIONARY[x+str(motTestCounter)+"." + a]=b
                   elif type(bb)==dict:
                      for aa, bbb in bb.items():
                           OUTPUTDICTIONARY[x+str(motTestCounter)+"." + a+str(an)+"." + aa]=bbb
                           
              
fname=mot_history['registration'] + '.xlsx'
OUTPUTDATAFRAME = pd.DataFrame.from_dict(OUTPUTDICTIONARY,orient='index')
OUTPUTDATAFRAME.to_excel(fname)
#print(d)


In [23]:
for x, y in mot_history.items():
  print(x,y)

for x in thisdict.values():

registration SD10KNS
make LAND ROVER
model DISCOVERY
firstUsedDate 2010-03-30
fuelType Diesel
primaryColour Silver
registrationDate 2010-03-30
manufactureDate 2010-03-30
engineSize 2993
hasOutstandingRecall Unknown
motTests [{'motTestNumber': '393037649774', 'completedDate': '2024-08-30T11:27:56.000Z', 'expiryDate': '2025-09-05', 'odometerValue': '82891', 'odometerUnit': 'MI', 'odometerResultType': 'READ', 'testResult': 'PASSED', 'dataSource': 'DVSA', 'defects': []}, {'motTestNumber': '589751055654', 'completedDate': '2023-08-24T10:28:26.000Z', 'expiryDate': '2024-09-05', 'odometerValue': '74944', 'odometerUnit': 'MI', 'odometerResultType': 'READ', 'testResult': 'PASSED', 'dataSource': 'DVSA', 'defects': [{'dangerous': False, 'text': 'under body slightly corroded', 'type': 'ADVISORY'}]}, {'motTestNumber': '739586855345', 'completedDate': '2022-09-06T16:04:29.000Z', 'expiryDate': '2023-09-05', 'odometerValue': '74643', 'odometerUnit': 'MI', 'odometerResultType': 'READ', 'testResult': 'P

In [ ]:


for x, obj in mot_history.items():
  print(x)

  for y in obj:
    print(y + ':', obj[y])

In [24]:
# Using recursion to get all values from nested dictionary
def approach1Fn(d):
    val = []

    for v in d.values():
        if isinstance(v, dict):
            val.extend(approach1Fn(v))

        elif isinstance(v, list):
            for i in v:
                if isinstance(i, dict):
                    val.extend(approach1Fn(i))
                else:
                    val.append(i)
        else:
            val.append(v)
    return val


print(approach1Fn(mot_history))

['SD10KNS', 'LAND ROVER', 'DISCOVERY', '2010-03-30', 'Diesel', 'Silver', '2010-03-30', '2010-03-30', '2993', 'Unknown', '393037649774', '2024-08-30T11:27:56.000Z', '2025-09-05', '82891', 'MI', 'READ', 'PASSED', 'DVSA', '589751055654', '2023-08-24T10:28:26.000Z', '2024-09-05', '74944', 'MI', 'READ', 'PASSED', 'DVSA', False, 'under body slightly corroded', 'ADVISORY', '739586855345', '2022-09-06T16:04:29.000Z', '2023-09-05', '74643', 'MI', 'READ', 'PASSED', 'DVSA', False, 'Nearside Front Inner Drive shaft joint constant velocity boot severely deteriorated (6.1.7 (g) (i))', 'MINOR', False, 'underside has surface corrosion', 'ADVISORY', False, 'Front Brake disc worn, pitted or scored, but not seriously weakened both (1.1.14 (a) (ii))', 'ADVISORY', False, 'Front Lower Suspension arm pin or bush worn but not resulting in excessive movement rear bush both sides (5.3.4 (a) (i))', 'ADVISORY', False, 'Rear Upper Suspension arm pin or bush worn but not resulting in excessive movement both sides (